In [1]:
import pandas as pd
import glob
allFiles = glob.glob("Events/*.csv")

In [2]:
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_)
    list_.append(df)
frame = pd.concat(list_)

In [3]:
frame = frame.drop('Unnamed: 0', axis=1)

In [4]:
frame = frame.reset_index().drop('index', axis=1)

In [5]:
frame['fromto'] = frame[['from','to']].apply(lambda x: '-'.join(x), axis=1)

In [6]:
import datetime
def short_month(shortMonth):
    return{
            'Jan' : 1,
            'Feb' : 2,
            'Mar' : 3,
            'Apr' : 4,
            'May' : 5,
            'Jun' : 6,
            'Jul' : 7,
            'Aug' : 8,
            'Sep' : 9, 
            'Oct' : 10,
            'Nov' : 11,
            'Dec' : 12
    }[shortMonth]


def long_format_parse(f):
    # parse from
    f = f.replace(',','')
    dtl = filter(None, f.split(' '))

    year = int(dtl[3])
    month = short_month(dtl[1])
    day = int(dtl[2])
    day_night = 0
    if dtl[-1].upper() == 'P' or dtl[-1].upper() == ' P':
        day_night = 1
    
    hour = 0
    minute = 0
    
    try:
        times = dtl[4][0:-1].split(':')
        hour = int(times[0]) + (12 * day_night)
        minute = int(times[1])
    except:
        pass
    
    x = datetime.datetime(year,month,day,hour,minute)
    return x

def short_format_parse(f, year, month, day):
    day_night = 0
    if f[-1].upper() == 'P' or f[-1].upper() == ' P':
        day_night = 1
    
    times = f[0:-1].split(':')
    hour , minute = 0 , 0
    add_day = False
    
    if len(times) == 2:
        hour = int(times[0]) + (12 * day_night)
        if hour >= 24:
            hour = hour % 24
            add_day = True
        minute = int(times[1])
        
    elif len(times) == 1:
        hour = int(times[0]) + (12 * day_night)
        
    x = datetime.datetime(year,month,day,hour,minute)
    if add_day:
        x = x + datetime.timedelta(days=1)
    return x

def custom_parse_dates(dt):
    dt_r = dt.split('-')
    f = dt_r[0]
    t = dt_r[1]
    from_datetime = long_format_parse(f)
    to_datetime = ''
    if len(t) > 10:
        to_datetime = long_format_parse(t)
    else:
        to_datetime = short_format_parse(t, from_datetime.year, from_datetime.month, from_datetime.day)
    
    duration = to_datetime - from_datetime
    
    return (from_datetime, to_datetime, duration)
    

In [7]:
frame = pd.concat([frame, frame['fromto'].apply(lambda x: pd.Series(custom_parse_dates(x)))],axis = 1)

In [8]:
frame.head()

,event,location,from,to,admision,address,category,fromto,0,1,2
0,Allston Christmas,Curbs and sidewalks,"Tuesday, Sep 01, 2015 5:00a",11:30p,FREE,Allston and surrounding neighborhoods Allston ...,"Alcohol, Meetup, Party, University","Tuesday, Sep 01, 2015 5:00a-11:30p",2015-09-01 05:00:00,2015-09-01 23:30:00,0 days 18:30:00
1,Fall Fashion Giveaway at the MGH Blood Donor C...,MGH Blood Donor Center,"Tuesday, Sep 01, 2015 7:30a","Wednesday, Sep 30, 2015 5:00p",FREE,"55 Fruit Street - Jackson 120 Boston , MA 02114",Social Good,"Tuesday, Sep 01, 2015 7:30a-Wednesday, Sep 3...",2015-09-01 07:30:00,2015-09-30 05:00:00,28 days 21:30:00
2,Toddler Tuesdays,Spectacle Island,"Tuesday, Sep 01, 2015 10:00a",10:30a,FREE,"66 Long Wharf Boston , MA 02110",Kid Friendly,"Tuesday, Sep 01, 2015 10:00a-10:30a",2015-09-01 10:00:00,2015-09-01 10:30:00,0 days 00:30:00
3,Storytelling,Faneuil Hall Marketplace,"Tuesday, Sep 01, 2015 10:00a",10:40a,FREE,"4 South Market Street Boston , MA 02109","Kid Friendly, Music, Performing Arts","Tuesday, Sep 01, 2015 10:00a-10:40a",2015-09-01 10:00:00,2015-09-01 10:40:00,0 days 00:40:00
4,"Integral Resource,Inc. Recruitment Session at ...",JVS CareerSolution,"Tuesday, Sep 01, 2015 11:00a",2:00p,FREE,"75 Federal Street, 3rd Floor Boston , Massachu...",Business,"Tuesday, Sep 01, 2015 11:00a-2:00p",2015-09-01 11:00:00,2015-09-01 14:00:00,0 days 03:00:00


In [9]:
frame = frame.drop(['from','to','fromto'], axis = 1)

In [11]:
frame.columns

Index([u'event', u'location', u'admision', u'address', u'category', 0, 1, 2], dtype='object')

In [12]:
frame.columns = [u'event', u'location', u'admision', u'address', u'category', u'from', u'to', u'duration']

In [13]:
frame = frame.sort('from')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [14]:
frame = frame.drop_duplicates()

In [15]:
frame = frame.reset_index().drop('index',axis=1)

In [16]:
frame.dtypes

event                object
location             object
admision             object
address              object
category             object
from         datetime64[ns]
to           datetime64[ns]
duration    timedelta64[ns]
dtype: object

In [17]:
#unique category
tranpose = frame['category'].apply(lambda x: pd.Series([i for i in reversed(str(x).split(','))]))

In [18]:
import numpy as np
category_list = np.unique(tranpose)

/usr/local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [19]:
category_list = category_list[1:].tolist() #remove nan

In [20]:
def categorize(category_data):
    result = np.zeros(len(category_list))
    if isinstance(category_data, basestring):
        cal_list = filter(None, category_data.split(','))
        for cal in cal_list:
            result[category_list.index(cal)] = 1
    return pd.Series(result)

In [21]:
frame = pd.concat([frame, frame['category'].apply(lambda x: categorize(x))], axis=1)

In [27]:
new_col = frame.columns[0:8].tolist()

In [28]:
new_col = new_col + map(str.strip,category_list)

In [31]:
frame.columns = new_col

In [32]:
frame.to_csv('Events/boston_events.csv')